In [4]:
import os
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from torch.utils.data import Dataset, DataLoader

import numpy as np
from PIL import Image

In [5]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 10
no_cuda = False
log_interval = 100

# Model

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Preprocess

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [8]:
# 압축 풀기
!unzip -qq /content/drive/"My Drive"/"[AOP:딥러닝 인공지능]"/TEST/"1. [Tensorflow 2.0 과 Pytorch]"/"3. 이미지 분석으로 배우는 tensorflow 2.0과 Pytorch"/"3. dataset"/mnist_png.zip


In [9]:
train_paths = glob('/content/mnist_png/training/*/*.png')[:1000]
test_paths = glob('/content/mnist_png/testing/*/*.png')[:1000]

len(train_paths), len(test_paths)

(1000, 1000)

참고: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class

In [13]:
class Dataset(Dataset):
    def __init__(self, data_paths, transform=None):

        self.data_paths = data_paths
        self.transform = transform

    def __len__(self):
        return len(self.data_paths)

    def __getitem__(self, idx):
        path = self.data_paths[idx]
        image = Image.open(path).convert("L")
        label = int(path.split('/')[-2]) # 맥의 경우 경로
        #label = int(path.split('\\')[-2]) # 윈도
        
        if self.transform:
            image = self.transform(image)

        return image, label

In [14]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    Dataset(train_paths, 
            transforms.Compose([
                transforms.RandomHorizontalFlip(), 
                transforms.ToTensor(), 
                transforms.Normalize(
                    mean=[0.406], 
                    std=[0.225])])
           ),
    batch_size=batch_size, 
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    Dataset(test_paths,
           transforms.Compose([
               transforms.ToTensor(), 
               transforms.Normalize(
                   mean=[0.406], 
                   std=[0.225])])
           ),
    batch_size=batch_size, 
    shuffle=False
)

In [15]:
for i, data in enumerate(train_loader):
    if i == 0:
        print(data[0].shape, data[1].shape)
        break

torch.Size([64, 1, 28, 28]) torch.Size([64])


# Optimization

In [16]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

# Learning Rate Scheduler

In [17]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [18]:
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=0, verbose=True) 

# Training

In [19]:
for epoch in range(1, epochs + 1):
    # Train Mode
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)  # https://pytorch.org/docs/stable/nn.html#nll-loss
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
    
    
    # Test mode
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    
    accuracy = 100. * correct / len(test_loader.dataset)
    
    # Learning Rate Scheduler
    scheduler.step(accuracy, epoch)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))

Train Epoch: 1 [0/1000 (0%)]	Loss: 2.445343


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:603: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)



Test set: Average loss: 0.8168, Accuracy: 1000/1000 (100%)

Train Epoch: 2 [0/1000 (0%)]	Loss: 0.783363
Epoch     2: reducing learning rate of group 0 to 1.0000e-04.

Test set: Average loss: 0.1059, Accuracy: 1000/1000 (100%)

Train Epoch: 3 [0/1000 (0%)]	Loss: 0.094200
Epoch     3: reducing learning rate of group 0 to 1.0000e-05.

Test set: Average loss: 0.0940, Accuracy: 1000/1000 (100%)

Train Epoch: 4 [0/1000 (0%)]	Loss: 0.084206
Epoch     4: reducing learning rate of group 0 to 1.0000e-06.

Test set: Average loss: 0.0930, Accuracy: 1000/1000 (100%)

Train Epoch: 5 [0/1000 (0%)]	Loss: 0.082626
Epoch     5: reducing learning rate of group 0 to 1.0000e-07.

Test set: Average loss: 0.0929, Accuracy: 1000/1000 (100%)

Train Epoch: 6 [0/1000 (0%)]	Loss: 0.080813
Epoch     6: reducing learning rate of group 0 to 1.0000e-08.

Test set: Average loss: 0.0929, Accuracy: 1000/1000 (100%)

Train Epoch: 7 [0/1000 (0%)]	Loss: 0.083896

Test set: Average loss: 0.0929, Accuracy: 1000/1000 (100%)
